## IMPORTS and UTILS

In [2]:
print("hi")

import os
import pandas as pd
import subprocess
import re
import numpy as np
import json
import csv



import matplotlib.pyplot as plt


import os

from robust_evaluation_tools.robust_utils import get_site, get_metric, get_disease, add_nb_patients_and_diseased
from robust_evaluation_tools.robust_harmonization import fit, apply, QC, compare_with_compilation 
from robust_evaluation_tools.synthectic_sites_generations import generate_sites

SYNTHETIC_SITES = "IMPACT_SICK/SYNTHETIC_SITES"

MAINFOLDER = "IMPACT_SICK/RESULTS"

ANALYSISFOLDER = "IMPACT_SICK/ANALYSIS"

hi


## HARMONIZATION

In [2]:
def harmonize(f_train,f_test, ref_data_file, metric, harmonizartion_method,  directory, robust, rwp,hc):
    os.makedirs(directory, exist_ok=True)
    
    # Fit the model
    output_model_filename = fit(f_train, ref_data_file, metric, harmonizartion_method, robust, rwp, directory, hc,)
    # Apply the model
    output_filename = apply(f_test, output_model_filename, metric, harmonizartion_method, robust, rwp, directory)
    
    # Perform quality control
    dists, bundle_names = QC(ref_data_file, output_filename, output_model_filename)
    dists_df = pd.DataFrame([dists], columns=bundle_names)
    dists_df['site'] = get_site(f_train)
    dists_df['metric'] = get_metric(f_train)
    dists_df['disease'] = get_disease(f_train)

    mea = compare_with_compilation(pd.read_csv(output_filename))
    mea['site'] = get_site(f_train)
    mea['metric'] = get_metric(f_train)
    mea['disease'] = get_disease(f_train)
    
    return[dists_df, mea]

In [3]:
def analyse_site(f_train,f_test,ref_data_file,metric,harmonizartion_method, directory):
    # 4 harmonization
    harmonization_hc = harmonize(f_train, f_test, ref_data_file, metric, harmonizartion_method, os.path.join(directory, "hc"), "No", False, True)
    harmonization_sick = harmonize(f_train, f_test,ref_data_file, metric, harmonizartion_method, os.path.join(directory, "sick"), "No", False, False)

    distances_combined = pd.concat([harmonization_hc[0], harmonization_sick[0]], ignore_index=True)
    distances_combined['method'] = ['hc', 'no_robust']

    # Combine MEA in a single DataFrame
    mea_combined = pd.concat([harmonization_hc[1], harmonization_sick[1]], ignore_index=True)
    mea_combined['method'] = ['hc', 'no_robust']


    #TODO bundles et analyze outliers
    return distances_combined, mea_combined

In [4]:
    
#Analyse Method
def analyse_method(sample_sizes, disease_ratios, num_tests,metrics, harmonizartion_method, main_directory, SYNTHETIC_SITES_VERSION):
    # Split the data into training and testing sets
    directory_site = os.path.join(SYNTHETIC_SITES ,SYNTHETIC_SITES_VERSION)
    # Initialize DataFrames to store the results
    dists_compilation = pd.DataFrame()
    mea_compilation = pd.DataFrame()
    for disease in [d for d in sorted(os.listdir(directory_site)) if os.path.isdir(os.path.join(directory_site, d))]:
        directory = os.path.join(main_directory, disease)
        for sample_size in sample_sizes:
            for disease_ratio in disease_ratios:        
                sizeDir = os.path.join(directory, f"{sample_size}_{int(disease_ratio*100)}")
                sizeDir_site = os.path.join(directory_site,disease, f"{sample_size}_{int(disease_ratio*100)}")
                for i in range(num_tests):
                    for metric in metrics:
                        tempDir = os.path.join(sizeDir, f"{i}")
                        tempDir_site = os.path.join(sizeDir_site, f"{i}")
                        os.makedirs(tempDir, exist_ok=True)

                        train_file_name = f"train_{sample_size}_{int(disease_ratio*100)}_{i}_{metric}.csv"
                        test_file_name = f"test_{sample_size}_{int(disease_ratio*100)}_{i}_{metric}.csv"
                        
                        # Sauvegarder l'échantillon dans un fichier temporaire
                        temp_file = os.path.join(tempDir_site,train_file_name )
                        train_df = pd.read_csv(temp_file)
                        train_df.to_csv(os.path.join(tempDir,train_file_name ), index=False)

                        test_file = os.path.join(tempDir_site, test_file_name)
                        test_df = pd.read_csv(test_file)
                        test_df.to_csv(os.path.join(tempDir,test_file_name ), index=False)

                        ref_data_file = os.path.join('DONNES','CamCAN', f"CamCAN.{metric}.raw.csv.gz")
                    
                        # Analyser le site pour le nouvel échantillon
                        dists_analyze, mea_analyze= analyse_site(temp_file, test_file, ref_data_file,metric,harmonizartion_method, tempDir)
                        dists_compilation = pd.concat([dists_compilation, dists_analyze])
                        mea_compilation = pd.concat([mea_compilation, mea_analyze])               
    dists_compilation.to_csv(os.path.join(directory, "dists_compilation.csv"), index=False)
    mea_compilation.to_csv(os.path.join(directory, "mea_compilation.csv"), index=False)

## EXECUTOR

In [5]:
harmonizartion_method= "classic"

SYNTHETIC_SITES_VERSION = "v1"
raw_directory = os.path.join('DONNES', 'COMPILATIONS')


sample_sizes = [40]  # Différentes tailles d'échantillon
disease_ratios = [0.1, 0.3, 0.5, 0.7]  # Différents pourcentages de malades
#sample_sizes = [150, 300]  # Différentes tailles d'échantillon
#disease_ratios = [0.3]  # Différents pourcentages de malades
num_tests = 8  # Nombre de tests à effectuer pour chaque combinaison
camcan_data = pd.read_csv('DONNES/CamCAN_combination_all_metrics.csv.gz')
metrics = ["ad", "adt", "afd", "fa", "fat", "fw", "md", "mdt", "rd", "rdt"]


for disease in [d for d in sorted(os.listdir(raw_directory)) if os.path.isdir(os.path.join(raw_directory, d))]:
    data_path = os.path.join(raw_directory,f'{disease}_combination_all_metrics.csv.gz')
    data = pd.read_csv(data_path)
    num_unique_sid_hc = data[data['disease'] == 'HC']['sid'].nunique()
    directory_site = os.path.join(SYNTHETIC_SITES ,SYNTHETIC_SITES_VERSION, disease)
    #generate_sites(sample_sizes, disease_ratios, num_tests, directory_site, data_path, disease=None)

#analyse_method(sample_sizes, disease_ratios, num_tests, metrics,harmonizartion_method, MAINFOLDER, SYNTHETIC_SITES_VERSION)

## ANALYSYS

In [22]:
# # Moyenne par site
# # Nothing really interesting so far

# directory = ANALYSISFOLDER
# os.makedirs(directory, exist_ok=True)

# dists_compilation['site'] = dists_compilation['site'].str.rsplit('_', n=1).str[0]
# metrics_compilation['site'] = metrics_compilation['site'].str.rsplit('_', n=1).str[0]

# # Display the means by site
# dists_means_by_site = dists_compilation.groupby(['site','method']).mean().reset_index()
# metrics_means_by_site = metrics_compilation.groupby(['site', 'metric']).mean().reset_index()

# metrics_means_by_site.to_csv(os.path.join(directory, "metrics_compilation_mean.csv"), index=False)
# dists_means_by_site.to_csv(os.path.join(directory, "dists_compilation_mean.csv"), index=False)
# print("FINI")

In [23]:

# # Exemple d'utilisation
# precision_df = calculate_precision_by_bundle(pd.read_csv(os.path.join(MAINFOLDER, robust_method, "metrics_compilation.csv")))
# precision_df = precision_df.sort_values(by='precision', ascending=False)
# precision_df.to_csv(os.path.join(directory, "metrics_per_bundle.csv"), index=False)

In [24]:
# precision_df = precision_df.sort_values(by='precision', ascending=False)
# precision_df.head()

In [25]:
# precision_df = precision_df.sort_values(by='f1_score', ascending=False)
# precision_df.head()

In [ ]:
# CREATION BOX PLOT POUR DISTANCES
def plot_bundle(df, sample_size, disease, metric, directory):
    """
    Crée un graphique pour chaque bundle dans le DataFrame donné.
    L'axe des X représente le nombre de patients et l'axe des Y représente la moyenne de la colonne du bundle.
    La courbe inclut une zone indiquant l'écart-type (std).

    Parameters:
    df (pd.DataFrame): Le DataFrame contenant les données.
    bundle_column (str): Le nom de la colonne du bundle à utiliser pour le graphique.
    """
    directory = os.path.join(directory, "DISTANCES_PLOTS", disease, metric, str(sample_size))
    df = df[df['num_patients'] == sample_size]
    df = df[df['disease'] == disease]
    df = df[df['metric'] == metric]
    os.makedirs(directory, exist_ok=True)
    
    for bundle_column in df.columns:
        if bundle_column in ['site','method','num_patients','disease_ratio','num_diseased', 'metric', 'disease']:
            continue # Skip non-numeric columns
        bundle_df = df[[bundle_column, 'site', 'method','num_patients','disease_ratio','num_diseased']].copy()
        methods = ["hc", "no_robust"]
        colors = ['green', 'red']
        
        plt.figure(figsize=(10, 5))
        width = 0.2  # the width of the bars
        x = np.arange(len(bundle_df['disease_ratio'].unique()))  # the label locations
        fig, ax = plt.subplots(figsize=(10, 5))

        for i, (method, color) in enumerate(zip(methods, colors)):
            method_df = bundle_df[bundle_df['method'] == method]
            data = [method_df[method_df['disease_ratio'] == patients][bundle_column].values 
                    for patients in bundle_df['disease_ratio'].unique()]
            
            # Ensure there is data for each num_patients
            if any(len(d) > 0 for d in data):
                positions = x + i * width  # Shift positions for each method
                ax.boxplot(data, positions=positions, widths=0.15, patch_artist=True, 
                        boxprops=dict(facecolor=color, color=color),
                        medianprops=dict(color='black'))
                
        ax.set_xlabel('Prct de patients malades')
        ax.set_ylabel('Distance de Battacharyya')
        ax.set_title(f'Boxplots pour le bundle: {bundle_column} avec {sample_size} de malades')
        ax.set_xticks(x + width * (len(methods) - 1) / 2)
        ax.set_xticklabels(bundle_df['disease_ratio'].unique())
        ax.legend(handles=[plt.Line2D([0], [0], color=color, lw=2, label=f'Method: {method}') for method, color in zip(methods, colors)])
        plt.savefig(os.path.join(directory, f'{bundle_column}_boxplot.png'))
        plt.close()
diseases = ["AD", "ADHD", "BIP", "MCI", "SCHZ", "TBI"]
# Exemple d'utilisation
distances_df = pd.read_csv(os.path.join(MAINFOLDER, "dists_compilation.csv"))
add_nb_patients_and_diseased(distances_df)
sample_sizes = [40]
for disease in diseases:
    for sample_size in sample_sizes:
        for metric in metrics:
            plot_bundle(distances_df, sample_size, disease, metric, ANALYSISFOLDER)

In [1]:
# CREATION BOX PLOT POUR MEA
def plot_mea(df, sample_size, disease, metric, directory):
    """
    Crée un graphique pour chaque bundle dans le DataFrame donné.
    L'axe des X représente le nombre de patients et l'axe des Y représente la moyenne de la colonne du bundle.
    La courbe inclut une zone indiquant l'écart-type (std).

    Parameters:
    df (pd.DataFrame): Le DataFrame contenant les données.
    bundle_column (str): Le nom de la colonne du bundle à utiliser pour le graphique.
    """
    directory = os.path.join(directory, "MEA_PLOTS", disease, metric, str(sample_size))
    df = df[df['num_patients'] == sample_size]
    df = df[df['disease'] == disease]
    df = df[df['metric'] == metric]
    os.makedirs(directory, exist_ok=True)
    
    for bundle_column in df.columns:
        if bundle_column in ['site','method','num_patients','disease_ratio','num_diseased', 'metric', 'disease']:
            continue # Skip non-numeric columns
        bundle_df = df[[bundle_column, 'site', 'method','num_patients','disease_ratio','num_diseased']].copy()
        methods = ["hc", "no_robust"]
        colors = ['green', 'red']
        
        plt.figure(figsize=(10, 5))
        width = 0.2  # the width of the bars
        x = np.arange(len(bundle_df['disease_ratio'].unique()))  # the label locations
        fig, ax = plt.subplots(figsize=(10, 5))

        for i, (method, color) in enumerate(zip(methods, colors)):
            method_df = bundle_df[bundle_df['method'] == method]
            data = [method_df[method_df['disease_ratio'] == patients][bundle_column].values 
                    for patients in bundle_df['disease_ratio'].unique()]
            
            # Ensure there is data for each num_patients
            if any(len(d) > 0 for d in data):
                positions = x + i * width  # Shift positions for each method
                ax.boxplot(data, positions=positions, widths=0.15, patch_artist=True, 
                        boxprops=dict(facecolor=color, color=color),
                        medianprops=dict(color='black'))
                
        ax.set_xlabel('Prct de patients malades')
        ax.set_ylabel('MEA')
        ax.set_title(f'Boxplots pour le bundle: {bundle_column} avec {sample_size} de malades')
        ax.set_xticks(x + width * (len(methods) - 1) / 2)
        ax.set_xticklabels(bundle_df['disease_ratio'].unique())
        ax.legend(handles=[plt.Line2D([0], [0], color=color, lw=2, label=f'Method: {method}') for method, color in zip(methods, colors)])
        plt.savefig(os.path.join(directory, f'{bundle_column}_boxplot.png'))
        plt.close()
diseases = ["AD", "ADHD", "BIP", "MCI", "SCHZ", "TBI"]
# Exemple d'utilisation
mea_df = pd.read_csv(os.path.join(MAINFOLDER, "mea_compilation.csv"))
add_nb_patients_and_diseased(mea_df)
sample_sizes = [40]
for disease in diseases:
    for sample_size in sample_sizes:
        for metric in metrics:
            plot_mea(mea_df, sample_size, disease, metric, ANALYSISFOLDER)

NameError: name 'pd' is not defined

In [6]:
output_path = 'RESULTS/DISTRIBUTION_RESULTS/distance_metrics_results.csv'
distance_metrics_results = pd.read_csv(output_path)

average_results = distance_metrics_results.groupby(['disease', 'metric']).mean(numeric_only=True).reset_index()
average_results
highest_lowest_metrics = average_results.groupby('disease').agg(
    highest_bhattacharyya=('bhattacharyya_distance', lambda x: average_results.loc[x.idxmax(), 'metric']),
    lowest_bhattacharyya=('bhattacharyya_distance', lambda x: average_results.loc[x.idxmin(), 'metric']),
    highest_kl=('kl_divergence', lambda x: average_results.loc[x.idxmax(), 'metric']),
    lowest_kl=('kl_divergence', lambda x: average_results.loc[x.idxmin(), 'metric']),
    highest_euclidean=('euclidean_distance', lambda x: average_results.loc[x.idxmax(), 'metric']),
    lowest_euclidean=('euclidean_distance', lambda x: average_results.loc[x.idxmin(), 'metric']),
    highest_mahalanobis=('mahalanobis_distance', lambda x: average_results.loc[x.idxmax(), 'metric']),
    lowest_mahalanobis=('mahalanobis_distance', lambda x: average_results.loc[x.idxmin(), 'metric'])
).reset_index()

highest_lowest_metrics

,disease,highest_bhattacharyya,lowest_bhattacharyya,highest_kl,lowest_kl,highest_euclidean,lowest_euclidean,highest_mahalanobis,lowest_mahalanobis
0,AD,fw,afd,mdt,afd,fw,rdt,fw,afd
1,ADHD,adt,fw,adt,fw,afd,mdt,afd,fw
2,BIP,adt,afd,adt,fw,fat,mdt,fat,mdt
3,MCI,fw,adt,adt,afd,fw,rdt,fw,afd
4,SCHZ,fat,md,adt,fw,fat,mdt,fa,mdt
5,TBI,afd,ad,afd,fa,afd,ad,afd,ad


In [8]:
output_path = 'RESULTS/DISTRIBUTION_RESULTS/distance_metrics_results.csv'
distance_metrics_results = pd.read_csv(output_path)

# Find the bundle/metric with the highest and lowest value for each distance metric for each disease
highest_lowest_values = distance_metrics_results.groupby('disease').agg(
    highest_bhattacharyya=('bhattacharyya_distance', lambda x: distance_metrics_results.loc[x.idxmax(), 'metric_bundle']),
    lowest_bhattacharyya=('bhattacharyya_distance', lambda x: distance_metrics_results.loc[x.idxmin(), 'metric_bundle']),
    highest_kl=('kl_divergence', lambda x: distance_metrics_results.loc[x.idxmax(), 'metric_bundle']),
    lowest_kl=('kl_divergence', lambda x: distance_metrics_results.loc[x.idxmin(), 'metric_bundle']),
    highest_euclidean=('euclidean_distance', lambda x: distance_metrics_results.loc[x.idxmax(), 'metric_bundle']),
    lowest_euclidean=('euclidean_distance', lambda x: distance_metrics_results.loc[x.idxmin(), 'metric_bundle']),
    highest_mahalanobis=('mahalanobis_distance', lambda x: distance_metrics_results.loc[x.idxmax(), 'metric_bundle']),
    lowest_mahalanobis=('mahalanobis_distance', lambda x: distance_metrics_results.loc[x.idxmin(), 'metric_bundle'])
).reset_index()

highest_lowest_values


,disease,highest_bhattacharyya,lowest_bhattacharyya,highest_kl,lowest_kl,highest_euclidean,lowest_euclidean,highest_mahalanobis,lowest_mahalanobis
0,AD,fw_mni_UF_L,fa_mni_ICP_R,rd_mni_CST_L,afd_mni_AC,fw_mni_F_L_R,rdt_mni_ICP_R,fw_mni_AC,afd_mni_MdLF_L
1,ADHD,fa_mni_ICP_L,fa_mni_OPT_R,mdt_mni_C_R,fw_mni_MdLF_R,afd_mni_ICP_L,mdt_mni_AF_L,afd_mni_MCP,fa_mni_OPT_R
2,BIP,fw_mni_ICP_R,rd_mni_F_L_R,adt_mni_AF_L,afd_mni_UF_R,fat_mni_F_L_R,mdt_mni_CCMid,fat_mni_CCMid,mdt_mni_ICP_R
3,MCI,fw_mni_ICP_R,afd_mni_ICP_R,adt_mni_PPT_L,fat_mni_AF_L,fw_mni_F_L_R,adt_mni_SLF_R,fw_mni_UF_L,afd_mni_FPT_L
4,SCHZ,fat_mni_CST_R,mdt_mni_CST_L,adt_mni_AF_R,fat_mni_MCP,fat_mni_IFOF_R,mdt_mni_IIT_mask_skeletonFA,fa_mni_IFOF_R,mdt_mni_IIT_mask_skeletonFA
5,TBI,afd_mni_SLF_R,ad_mni_FPT_R,afd_mni_OR_L,fa_mni_OPT_R,afd_mni_VOF_L,ad_mni_CST_R,afd_mni_IFOF_L,ad_mni_CST_R


## TESTS

In [ ]:
# # # TEST ADD BIAIS
# # Split the data into training and testing sets
# directory = os.path.join(MAINFOLDER, "testBiais")
# os.makedirs(directory, exist_ok=True)
# train_df, test_df = split_train_test(CAMCAN, test_size=0.2, random_state=42)

# # Generate biased data
# # Save the original non-biased data to temporary files
# temp_train_file_original = os.path.join(directory, "temp_train_original.csv")
# temp_test_file_original = os.path.join(directory, "temp_test_original.csv")
# train_df.to_csv(temp_train_file_original, index=False)
# test_df.to_csv(temp_test_file_original, index=False)

# # Generate biased data
# sampled_df_biaied, test_df_biaised, gammas,deltas, ruffles= generate_biaised_data(train_df, test_df)

# # Save the biased data to temporary files
# temp_train_file = os.path.join(directory, "temp_train_biased.csv")
# temp_test_file = os.path.join(directory, "temp_test_biased.csv")
# sampled_df_biaied.to_csv(temp_train_file, index=False)
# test_df_biaised.to_csv(temp_test_file, index=False)

# # Run the combat_visualize_data script
# outname_train = os.path.join("visualize_train")
# cmd = (
#     "scripts/combat_visualize_data.py"
#     + " "
#     + temp_train_file_original
#     + " "
#     + temp_train_file
#     + " --out_dir "
#     + directory
#     + " --outname "
#     + outname_train
#     + " -f"
#     + " --bundles all"
# )
# subprocess.call(cmd, shell=True)

# # Display gammas and deltas along with their mean and standard deviation
# print("Gammas:", gammas)
# print("Deltas:", deltas)
# gammas = list(gammas.values())
# deltas = list(deltas.values())
# print("\nGamma Statistics:")
# print(f"Mean: {np.mean(gammas)}, Std: {np.std(gammas)}")

# print("\nDelta Statistics:")
# print(f"Mean: {np.mean(deltas)}, Std: {np.std(deltas)}")
# print("Ruffles:", ruffles)


In [ ]:
# # TEST Powerpoint generation
# d  = os.path.join(MAINFOLDER, robust_method, "adni_100_Philips_3T")
# create_presentation(d)

In [ ]:
# # TEST the sample_patients function with compilation data data
# sampled_df = sample_patients(COMPILATION, num_patients=100, disease_ratio=0.5)
# print(sampled_df)


In [ ]:
# load_metrics("ROBUST/IQR/50_30/0/", "50_patients_30_percent_0")

In [ ]:
# # Load the dists_compilation and metrics_compilation CSV files
# dists_compilation_path = os.path.join(directory, "dists_compilation.csv")
# metrics_compilation_path = os.path.join(directory, "metrics_compilation.csv")

# dists_compilation = pd.read_csv(dists_compilation_path)
# metrics_compilation = pd.read_csv(metrics_compilation_path)

# # Change the site column
# dists_compilation['site'] = dists_compilation['site'].str.rsplit('_', n=1).str[0]
# metrics_compilation['site'] = metrics_compilation['site'].str.rsplit('_', n=1).str[0]

# # Display the means by site
# dists_means_by_site = dists_compilation.groupby(['site','comparaison']).mean()
# metrics_means_by_site = metrics_compilation.groupby('site').mean()

# print(dists_means_by_site)
# print(metrics_means_by_site)

In [ ]:
# # FIX METRICS COMPILATION
# directory = os.path.join(MAINFOLDER, robust_method)
# df = pd.read_csv(os.path.join(directory, "metrics_compilation.csv"))

# # Group by the site
# grouped = df.groupby('site')

# # Process each site
# cleaned_dfs = []
# for site, group in grouped:
#     # Reset index for easier manipulation
#     group = group.reset_index(drop=True)
    
#     # # The first row is the "bundle row" (new column names)
#     # new_columns = group.iloc[0].values  # Extract column names from the first row
#     # new_columns[-1] = 'site'
#     # group = group.iloc[1:]  # Remove the first row
    
#     # # Assign new column names
#     # group.columns = new_columns
    
#     # # Sort the columns alphabetically (excluding 'site')
#     # sorted = group.sort_index(axis=1)
#     # Add a new column 'nomm' with the value indicating the metric for each row
#     metrics = ['tp', 'fp', 'tn', 'fn', 'precision', 'recall', 'taux_faux_positifs', 'f1_score']
#     group['metric'] = metrics
    
#     # # Append the cleaned DataFrame for this site
#     cleaned_dfs.append(group)

# # Concatenate all cleaned DataFrames
# final_df = pd.concat(cleaned_dfs, ignore_index=True)

# # Save or display the result
# final_df.to_csv(os.path.join(directory, "metrics_compilation.csv"), index=False)


In [ ]:
# REAL SITES
# directory = os.path.join(MAINFOLDER, robust_method)
# raw_directory = os.path.join(RAWFOLDER, site_group)
# for filename in sorted(os.listdir(raw_directory)):
#     f = os.path.join(raw_directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         analyse_site(f, robust_method, directory)
        


In [ ]:
# ADD METRIC AND DIEASE COLUMNS
# dists_compilation = pd.read_csv(os.path.join(MAINFOLDER, "dists_compilation.csv"))
# mea_compilation = pd.read_csv(os.path.join(MAINFOLDER, "mea_compilation.csv"))
# # List of metrics and diseases
# metrics = ['ad', 'adt', 'afd', 'fa', 'fat', 'fw', 'md', 'mdt', 'rd', 'rdt']
# diseases = ["AD", "ADHD", "BIP", "MCI", "SCHZ", "TBI"]
# # Repeat each metric 2 times and then switch to the next metric
# metrics_repeated = [metric for metric in metrics for _ in range(2)]

# # Repeat the entire metrics list twice for each disease
# metrics_final = metrics_repeated * 8 * len(diseases)

# # Repeat each disease for the length of the metrics list repeated twice
# diseases_final = [disease for disease in diseases for _ in range(len(metrics_repeated) * 8)]

# # Add the new columns to the DataFrame
# dists_compilation['metric'] = metrics_final[:len(dists_compilation)]
# dists_compilation['disease'] = diseases_final[:len(dists_compilation)]

# mea_compilation['metric'] = metrics_final[:len(mea_compilation)]
# mea_compilation['disease'] = diseases_final[:len(mea_compilation)]